In [1]:
import pandas as pd
import numpy as np
import pickle 
from tqdm import tqdm 
from collections import Counter,OrderedDict
from sklearn.metrics.pairwise import cosine_similarity
import nltk 
from nltk.corpus import stopwords
import string  

nltk.download("stopwords")

def flatten_comprehension(matrix):
    #https://realpython.com/python-flatten-list/
    return [item for row in matrix for item in row]

# Load data
behaviors = pd.read_csv('./small_training_data/behaviors.tsv', delimiter='\t', header=None)
news = pd.read_csv('./small_training_data/news.tsv', delimiter='\t', header=None)

# Naming columns
behaviors.columns = ["impression_id", "user_id", "time", "history", "impressions"]
news.columns = ["news_id", "category", "subcategory", "title", "abstract", "url", "title_entities", "abstract_entities"]


# Extracting clicked news from behaviors, this is a column of lists of the clicked news (tagget with 1) for each impression
behaviors['clicked_news'] = behaviors['impressions'].apply(lambda x: [imp.split('-')[0] for imp in x.split() if imp.split('-')[1] == '1'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hal9000/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#define unique users and their read articles
unique_users_series = behaviors["user_id"].drop_duplicates()
unique_users_dict = {}  #user_id coupled to news_articles

#aggregate all read articles and couple it to the user in the unique_users_frame
i = 0
for user_id in tqdm(unique_users_series,ascii=True):
    user_specific_behavior = behaviors[behaviors["user_id"] == user_id]
    try:
        history_list = flatten_comprehension([elem.split(" ") for elem in user_specific_behavior["history"]])
    except:
        pass 
    clicked_list = flatten_comprehension([elem for elem in user_specific_behavior["clicked_news"]])
    all_read_articles = history_list + clicked_list
    unique_users_dict[user_id] = all_read_articles


with open('unique_users.pkl', 'wb') as file: #serialize so we don't have to do this inefficient code again
    pickle.dump(unique_users_dict,file)



100%|####################################################################################################################| 50000/50000 [03:52<00:00, 215.03it/s]


In [3]:
#define the embedding. (what words we look for)

def remove_punctuation(sentence:str) -> str:
    sentence = sentence.replace("'","")
    for elem in string.punctuation:
        sentence = sentence.replace(elem," ")
    return sentence

def remove_stopwords(sentence:str)-> str:
    stop_words = list(set(stopwords.words("english")))
    sentence= [ elem for elem in sentence.split(" ") if elem not in stop_words]
        
    return " ".join(sentence)

def preprocess(sentence:str) -> Counter:
    counter = Counter(remove_punctuation(remove_stopwords(sentence.lower())).split(" "))
    counter[''] =0 
    return counter


# total_title = ""
# for title in tqdm(news["title"],ascii=True):
#     total_title += " "+title 
# total_BOW = preprocess(total_title)


# sorted_BOW = sorted(OrderedDict(total_BOW).items(),key=lambda x: x[1],reverse=True)
# print(sorted_BOW)
    

100%|################################################################################################################| 51282/51282 [00:00<00:00, 2701791.18it/s]


[('new', 2353), ('trump', 1651), ('man', 1469), ('says', 1402), ('week', 1376), ('police', 1329), ('year', 1224), ('state', 1157), ('game', 1092), ('2', 1054), ('best', 1043), ('2019', 1008), ('first', 977), ('vs', 975), ('1', 952), ('school', 947), ('home', 928), ('football', 926), ('day', 900), ('3', 881), ('fire', 872), ('impeachment', 861), ('win', 854), ('old', 832), ('report', 828), ('county', 812), ('crash', 812), ('top', 811), ('10', 808), ('one', 797), ('get', 784), ('5', 782), ('say', 769), ('woman', 749), ('season', 744), ('2020', 742), ('could', 728), ('back', 723), ('4', 703), ('high', 700), ('killed', 678), ('time', 677), ('watch', 664), ('world', 635), ('city', 630), ('make', 627), ('shooting', 626), ('heres', 602), ('nfl', 601), ('years', 600), ('us', 589), ('house', 589), ('s', 581), ('car', 577), ('i', 569), ('found', 559), ('u', 543), ('no', 536), ('news', 533), ('former', 522), ('7', 513), ('family', 509), ('two', 501), ('weather', 495), ('florida', 481), ('patriots

In [5]:
# embedding_words = [elem[0] for elem in sorted_BOW[:1000]]

with open('unique_users.pkl', 'rb') as file:
    users = pickle.load(file)

user_BOW = {} #{"user_id":embedding,etc.}

#compute user embeddings
for user in tqdm(users,ascii=True):
    news_ids = users[user]    
    counter =preprocess(news["title"][news["news_id"].isin(news_ids)].map(str.lower).apply(lambda x: x +" ").agg("sum"))
    
    user_BOW[user] = counter
    
with open('userBOW.pkl', 'wb') as file: #serialize so we don't have to do this inefficient code again
    pickle.dump(user_BOW,file)
    

100%|####################################################################################################################| 50000/50000 [01:29<00:00, 557.77it/s]


In [15]:
with open('userBOW.pkl', 'rb') as file:
    user_BOW = pickle.load(file)

df = pd.DataFrame()

for user1 in tqdm(user_BOW,ascii=True):
    for user2 in user_BOW:
        df_row = pd.DataFrame({"user1":[user1],"user2":[user2],"similarty":[1]})
        print(pd.concat([df, df_row], axis=1, join="inner")) 
    break 



  0%|                                                                                                                                 | 0/50000 [00:14<?, ?it/s]

Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns: [user1, user2, similarty]
Index: []
Empty DataFrame
Columns:

KeyboardInterrupt: 

In [ ]:
"""
NOTE 

instead of cosine similarity we use Jaccard similarity J (https://stats.stackexchange.com/questions/289400/quantify-the-similarity-of-bags-of-words)
because the BOW embeddings are really sparse except if you use to common words ( "a", "is", "of",etc) 
This would be counter intuitive to use. 

"""